# Homework 3: Reshaping, sorting (5)

In [1]:
import pandas as pd
import numpy as np

Some functions we have learned

- pd.Series() 
- df=pd.DataFrame({'A': 1., 'B': pd.Timestamp('20130102'), 'C': pd.Series(1, index=list(range(4)), dtype='float32'), 'D': np.array([3] * 4, dtype='int32'), 'E': pd.Categorical(["test", "train", "test", "train"]), 'F': 'foo'}) 
- df.T 
- df.index 
- df.columns 
- df.sort_values(by='C') 
- df['A'] 
- df.describe(include=['number', 'category', 'all'], percentiles=[.05, .25, .75, .95]) 
- df.max(skipna=False) 
- df.min() 
- df.sum() 
- df.mean() 
- df.unique() 
- df.idxmax() 
- df.iloc 
- df.value_counts() 
- df[['A', 'B']] (df['A']>1) & (df['B']>1) df[df['A']>1, 'B'] 
- df.loc[df['A']==3.14, 'A']=0 df.sort_values(by='A') 
- df.pivot_table(values='Name', aggfunc=np.sum, index='Row', columns='Column') 
- df.melt(id_vars=None, value_vars=None, var_name=None, value_name='value')

In [2]:
csv_path = 'https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv'
titanic = pd.read_csv(csv_path)
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Pivot tables

**Exercise 1:**  Generate a table using titanic.pivot_table(...) named Survived that has the Pclass as rows and Sex as columns for those that have survived.  Hint:  since the 'Survived' is 0 if they did not survive and 1 if they did, we can simply sum it for counting survived.

In [3]:
Survived=titanic.pivot_table(values='Survived', index='Pclass', columns='Sex', aggfunc=np.sum)
Survived

Sex,female,male
Pclass,,
1,91,45
2,70,17
3,72,47


**Exercise 2:**  Generate a table using titanic.pivot_table(...) named Total that has the Pclass as rows and Sex as columns for all passengers.  len is a perfectly acceptable aggfunc

In [4]:
Total=titanic.pivot_table(values='Survived', index='Pclass', columns='Sex', aggfunc=len)
Total

Sex,female,male
Pclass,,
1,94,122
2,76,108
3,144,347


If tables are exactly the same dimensions, we can actually just add/multiply etc with them.  If your Survived and Total work the table below should show the infamous survival rate by class/sex.

In [5]:
Survived/Total * 100

Sex,female,male
Pclass,,
1,96.808511,36.885246
2,92.105263,15.740741
3,50.000000,13.544669


## More pivot tables

Before we tried to look at the relationship between number the embarkation point and the first class passengers on the Titanic.  It took us 3 long lines to get the percentage of 1st class for each location.

In [6]:
print(len(titanic.loc[(titanic['Pclass']==1) & (titanic['Embarked']=='C')])/len(titanic[(titanic['Embarked']=='C')]))
print(len(titanic.loc[(titanic['Pclass']==1) & (titanic['Embarked']=='Q')])/len(titanic[(titanic['Embarked']=='Q')]))
print(len(titanic.loc[(titanic['Pclass']==1) & (titanic['Embarked']=='S')])/len(titanic[(titanic['Embarked']=='S')]))


0.5059523809523809
0.025974025974025976
0.1972049689440994


**Exercise 3:**   Instead of percentages, use pivot_table to generate a new table called EmbarkedClass that shows the # of each class by embarkation point.  You should have embarkation site in the rows of the new table and the passenger class in the columns.  

In [7]:
EmbarkedClass=titanic.pivot_table(values='Name', aggfunc=len, index='Embarked', columns='Pclass')
EmbarkedClass

Pclass,1,2,3
Embarked,,,
C,85,17,66
Q,2,3,72
S,127,164,353


If it worked well, the following should give the percentage of passengers at each embarkation point that correspond to each class.

DO NOT DELETE FOLLOWING CODE

In [8]:
EmbarkedClassPercents = EmbarkedClass.transform(lambda x: x/x.sum(), axis=1)*100
EmbarkedClassPercents

Pclass,1,2,3
Embarked,,,
C,50.595238,10.119048,39.285714
Q,2.597403,3.896104,93.506494
S,19.720497,25.465839,54.813665


**Exercise 4:** Take EmbarkedClassPercents and sort it so that Embarked location with the highest percentage of 3rd class is at the top and the lowest is at the bottom.  

In [9]:
EmbarkedClassPercents.sort_values(by=3, ascending=False)

Pclass,1,2,3
Embarked,,,
Q,2.597403,3.896104,93.506494
S,19.720497,25.465839,54.813665
C,50.595238,10.119048,39.285714


**Exercise 5:** Use melt on EmbarkedClass to get back a table with 3 columns:  Embarked, Pclass, and Counts.  Where Embarked is the C/Q/S code, Pclass is 1 or 2 (we are throwing out 3rd class) and Counts is the number of people associated with that pair.  Hint:  melt has a value_vars argument that lets you melt only some of the columns.

In [10]:
EmbarkedClass.reset_index().melt(id_vars='Embarked', value_vars=[1,2], value_name='Counts')

,Embarked,Pclass,Counts
0,C,1,85
1,Q,1,2
2,S,1,127
3,C,2,17
4,Q,2,3
5,S,2,164
